## Week 2 Day 3 - OpenAI Edition

Now we get to more detail using different OpenAI models:

1. Different OpenAI models (GPT-4o, GPT-4o-mini, GPT-3.5-turbo)

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import requests
import os
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
resend_api_key = os.getenv('RESEND_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if resend_api_key:
    print(f"Resend API Key exists and begins {resend_api_key[:2]}")
else:
    print("Resend API Key not set")

print("\n✅ Using OpenAI models only for this comparison!")


OpenAI API Key exists and begins sk-proj-
Resend API Key exists and begins re

✅ Using OpenAI models only for this comparison!


### Comparing Different OpenAI Models

Instead of using different AI providers, we'll compare different OpenAI models:
- **GPT-4o**: The most capable and latest model
- **GPT-4o-mini**: Faster and more cost-effective
- **GPT-3.5-turbo**: More basic but very fast

Each model has different strengths and characteristics that will show in the email generation.

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# Create OpenAI clients for different models
openai_client = AsyncOpenAI(api_key=openai_api_key)

# Different OpenAI models with different capabilities
gpt4o_model = OpenAIChatCompletionsModel(model="gpt-4o", openai_client=openai_client)
gpt4o_mini_model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
gpt35_turbo_model = OpenAIChatCompletionsModel(model="gpt-3.5-turbo", openai_client=openai_client)

print("OpenAI models configured:")
print("GPT-4o: Most capable, best reasoning")
print("GPT-4o-mini: Fast and efficient")
print("GPT-3.5-turbo: Quick and direct")


OpenAI models configured:
GPT-4o: Most capable, best reasoning
GPT-4o-mini: Fast and efficient
GPT-3.5-turbo: Quick and direct


In [6]:
sales_agent1 = Agent(name="GPT-4o Sales Agent", instructions=instructions1, model=gpt4o_model)
sales_agent2 = Agent(name="GPT-4o-mini Sales Agent", instructions=instructions2, model=gpt4o_mini_model)
sales_agent3 = Agent(name="GPT-3.5-turbo Sales Agent", instructions=instructions3, model=gpt35_turbo_model)

print("Created 3 sales agents with different OpenAI models:")
print(f"   1. {sales_agent1.name} - Professional style")
print(f"   2. {sales_agent2.name} - Humorous style")
print(f"   3. {sales_agent3.name} - Concise style")


Created 3 sales agents with different OpenAI models:
   1. GPT-4o Sales Agent - Professional style
   2. GPT-4o-mini Sales Agent - Humorous style
   3. GPT-3.5-turbo Sales Agent - Concise style


In [7]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

print("Created agent tools for each OpenAI model")


Created agent tools for each OpenAI model


In [8]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send out an email with the given subject and HTML body 
    to all sales prospects using Resend
    """
    
    # Get email addresses from environment variables
    from_email = os.getenv("FROM_EMAIL", "onboarding@resend.dev")
    to_email = os.getenv("TO_EMAIL", "alexjustdata@gmail.com")
    
    # Get the Resend API key from environment variable
    api_key = os.getenv("RESEND_API_KEY")
    
    # Validate that RESEND_API_KEY is available
    if not api_key:
        return {"status": "failure", 
                "message": "RESEND_API_KEY not found in environment variables"}
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Ensure proper HTML formatting if html_body is plain text
    if not html_body.strip().startswith('<'):
        # If it's plain text, convert line breaks to HTML
        formatted_html = html_body.replace('\n', '<br>')
        formatted_html = f"<div style='font-family: Arial, sans-serif; line-height: 1.6;'>{formatted_html}</div>"
    else:
        # If it's already HTML, use as is
        formatted_html = html_body
    
    payload = {
        "from": f"Alex <{from_email}>",
        "to": [to_email],
        "subject": subject,
        "html": formatted_html
    }
    
    try:
        response = requests.post(
            "https://api.resend.com/emails", 
            json=payload, 
            headers=headers
            )
        
        # Add debugging information
        print(f"Request payload: {payload}")
        print(f"Response status: {response.status_code}")
        print(f"Response body: {response.text}")
        
        if response.status_code == 200 or response.status_code == 202:
            return {"status": "success", 
                    "message": "HTML email sent successfully", 
                    "response": response.text}
        else:
            return {"status": "failure", 
                    "message": response.text, 
                    "status_code": response.status_code}
            
    except Exception as e:
        return {"status": "error", 
                "message": f"Exception occurred: {str(e)}"}

print("📧 Email sending function configured with Resend")


📧 Email sending function configured with Resend


In [9]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Using GPT-4o-mini for utility agents (cost-effective)
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

print("🎨 Created subject writer and HTML converter agents (GPT-4o-mini)")


🎨 Created subject writer and HTML converter agents (GPT-4o-mini)


In [10]:
email_tools = [subject_tool, 
               html_tool, 
               send_html_email]

In [11]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

print("📬 Email Manager agent created (GPT-4o-mini)")


📬 Email Manager agent created (GPT-4o-mini)


In [12]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]


In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

# Using GPT-4o for the sales manager (best reasoning for decision-making)
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

print("🎯 Sales Manager created with GPT-4o for best decision-making")
print("\n🚀 Running multi-model comparison with OpenAI models...")

with trace("OpenAI Multi-Model SDR"):
    result = await Runner.run(sales_manager, message)


🎯 Sales Manager created with GPT-4o for best decision-making

🚀 Running multi-model comparison with OpenAI models...


## Check out the trace:

https://platform.openai.com/traces

**Notice the differences between OpenAI models:**
- **GPT-4o**: More sophisticated reasoning and detailed content
- **GPT-4o-mini**: Balanced performance and efficiency  
- **GPT-3.5-turbo**: Direct and concise approach

Each model brings different strengths to the sales email generation process!

In [18]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# Using GPT-4o-mini for guardrail (cost-effective for validation tasks)
guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

print("🛡️ Guardrail agent created with structured output (GPT-4o-mini)")


🛡️ Guardrail agent created with structured output (GPT-4o-mini)


In [19]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(
        output_info={"found_name": result.final_output},
        tripwire_triggered=is_name_in_message
    )

print("Name detection guardrail function created")


Name detection guardrail function created


In [24]:
careful_sales_manager = Agent(
    name="Protected Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o",  # Best model for protected operations
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

print("🛡️ Protected Sales Manager created with guardrails")
print("⚠️  Testing with message containing a name (Alice)...")

# Demonstrate guardrail protection by catching the expected exception
try:
    with trace("Protected OpenAI Multi-Model SDR - Name Detection"):
        result = await Runner.run(careful_sales_manager, message)
    print("✅ Email sent successfully")
    
except Exception as e:
    if "InputGuardrailTripwireTriggered" in str(type(e)):
        print("GUARDRAIL TRIGGERED!")
        print("The name detection guardrail is working correctly!")
        print(f"🛡️ Blocked execution because the message contains a personal name")
        print(f"📝 Message was: '{message}'")
        print("This is the expected behavior for security protection")
    else:
        print(f"❌ Unexpected error: {e}")
        raise e


🛡️ Protected Sales Manager created with guardrails
⚠️  Testing with message containing a name (Alice)...
GUARDRAIL TRIGGERED!
The name detection guardrail is working correctly!
🛡️ Blocked execution because the message contains a personal name
📝 Message was: 'Send out a cold sales email addressed to Dear CEO from Alice'
This is the expected behavior for security protection


## 🛡️ Guardrail Successfully Activated!

https://platform.openai.com/traces

**The guardrail detected the name "Alice" and correctly blocked the request!**

### **What just happened:**
1. ✅ **Name Detection**: The guardrail agent identified "Alice" as a personal name
2. 🚨 **Tripwire Triggered**: Security protection activated automatically  
3. 🛡️ **Request Blocked**: Prevented potential privacy/security issues
4. 📊 **Trace Generated**: You can see the guardrail logic in the OpenAI traces

This demonstrates how **input guardrails** provide robust protection by validating requests before execution.

In [25]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

print("✅ Testing with message without personal names...")
print(f"📝 Message: '{message}'")

try:
    with trace("Protected OpenAI Multi-Model SDR - No Names"):
        result = await Runner.run(careful_sales_manager, message)
    print("🎉 SUCCESS! Email workflow completed")
    print("✅ No guardrail triggered - message passed security validation")
    print("📧 Email was generated and sent successfully")
    
except Exception as e:
    if "InputGuardrailTripwireTriggered" in str(type(e)):
        print("🚨 Unexpected: Guardrail triggered on safe message")
        print("🔍 Check if there are any names in the message")
    else:
        print(f"❌ Error: {e}")
        raise e


✅ Testing with message without personal names...
📝 Message: 'Send out a cold sales email addressed to Dear CEO from Head of Business Development'
🎉 SUCCESS! Email workflow completed
✅ No guardrail triggered - message passed security validation
📧 Email was generated and sent successfully


## OpenAI Model Comparison Summary

### **Model Characteristics:**

**GPT-4o (Professional Agent)**
- **Strengths**: Most sophisticated reasoning, detailed analysis
- **Best for**: Complex decision-making, nuanced content
- **Email style**: Professional, well-structured, comprehensive

**GPT-4o-mini (Humorous Agent)**  
- **Strengths**: Balanced performance, cost-effective
- **Best for**: Most business applications, good creativity
- **Email style**: Engaging, creative, balanced

**GPT-3.5-turbo (Concise Agent)**
- **Strengths**: Fast response, direct approach
- **Best for**: Simple tasks, quick turnaround
- **Email style**: Concise, direct, to-the-point

### **Key Learnings:**
1. **Different OpenAI models excel at different tasks**
2. **Model selection impacts output quality and style**
3. **Guardrails work consistently across all models**
4. **Structured outputs ensure reliable data extraction**

**This demonstrates how you can optimize your AI applications by choosing the right OpenAI model for each specific task!**


<table style="margin: 0; text-align: left; width:100%">
    <tr>
            <h2 style="color:brown">Exercise</h2>
            <span style="color:brown">• Try different OpenAI model configurations (temperature, max_tokens)<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation<br/>• Compare response times and costs between models
            </span>
    </tr>
</table>